In [1]:
%pylab inline
import pandas as pd
import os
from textwrap import dedent

Populating the interactive namespace from numpy and matplotlib


In [2]:
assembly_common_name_map = {'mm10': 'mouse', 'hg38': 'human'}

sheet_name = 'mm10'
excel_df = pd.read_excel('../data/datasets_info.xlsx', sheet_name=sheet_name)


In [5]:
excel_df

,GEO,SRA,RIBO sample,RNA sample,read lengths,offsets,RIBO adapter,RNA adapter,RIBO strandedness,RNA strandedness
0,GSE22001,SRP003554,SRX026871,SRX026872,"28,29,30","12,12,12",TCGTATG,TCGTATG,forward,forward
1,GSE72064,SRP062407,SRX1149649,SRX1149668,"28,29,30,31","12,12,12,12",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
2,GSE84112,SRP078005,SRX1900396,SRX1900402,"26,27,28,29,30","12,12,12,12,12",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
3,GSE89011,SRP091889,SRX2255510,SRX2255511,"26,27,28,29,30","12,12,12,12,12",AGATCGGAAGAGCACACGTCT,AGATCGGAAGAGCACACGTCT,forward,forward
4,GSE102890,SRP115915,SRX3110803,SRX3110807,"29,30,31,32,33,34","12,12,12,13,13,13",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
5,GSE99787,SRP108787,SRX2893684,SRX2893692,"25,26,27,28","12,10,11,12",None,None,forward,reverse


In [6]:
output_dir = '/home/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/ribORF/offsets/mm10'

In [7]:
!mkdir -p /home/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/ribORF/offsets/mm10

In [8]:
offset_file_map = {}
for index, row in excel_df.iterrows():
    read_lengths = row['read lengths'].strip().split(',')
    offsets = row['offsets'].strip().split(',')
    srp = row['SRA']
    ribo_sample = row['RIBO sample']
    rna_sample = row['RIBO sample']
    ribo_outfile = os.path.join(output_dir, '{}_{}_{}.offset.txt'.format(srp, 'ribo', ribo_sample))
    rna_outfile = os.path.join(output_dir, '{}_{}_{}.offset.txt'.format(srp, 'rna', rna_sample))
    
    with open(rna_outfile, 'w') as fh:
        for read_length, offset in zip(read_lengths, offsets):
            fh.write('{}\t{}\n'.format(read_length, offset))
            
    with open(ribo_outfile, 'w') as fh:
        for read_length, offset in zip(read_lengths, offsets):
            fh.write('{}\t{}\n'.format(read_length, offset))
        
    offset_file_map['{}_mouse'.format(srp)] = ribo_outfile

In [9]:
input_dir_root = '/home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/'
create_config_dir = '/home/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/ribORF/configs'
candidate_orfs = '/home/cmb-panasas2/skchoudh/genomes/mm10/ribORF_annotation/candidateORF.genepred.txt'
for srp in os.listdir(input_dir_root):
    print(srp)
    if srp=='test' or srp == 'SRP063852_human_old':
        continue
    input_dir = os.path.join(input_dir_root, srp)
    offset_file = offset_file_map[srp]
    config_file = os.path.join(create_config_dir, srp+'.py')
    config = dedent("""
    INPUT_DIR = '{}'
    OFFSET_FILE = '{}'
    CANDIDATE_ORFS = '{}'
    """).format(input_dir, offset_file, candidate_orfs)
    with open(config_file, 'w') as fh:
        fh.write(config)
    

SRP003554_mouse
SRP062407_mouse
SRP078005_mouse
SRP091889_mouse
SRP115915_mouse
